In [3]:
import pandas as pd
import json

In [4]:
df = pd.read_csv("/data/raw_stock_indicator.csv")

In [5]:
def json_parser(x):
    try:
        return json.loads(x)["symbol"]
    except Exception as e:
        print(e)
        return ""

In [6]:
df["company_symbol"] = df["info_json"].map(lambda x: json_parser(x))  

In [7]:
df["company_symbol"].unique()

array(['LTIM.NS', '20MICRONS.NS', '21STCENMGM.NS', '3MINDIA.NS',
       '3PLAND.NS', '5PAISA.NS', '63MOONS.NS', 'A2ZINFRA.NS', 'AAKASH.NS',
       'AARON.NS', 'AARTIDRUGS.NS', 'AARTIIND.NS', 'AARVEEDEN.NS',
       'AARVI.NS', 'AAVAS.NS', 'ABAN.NS', 'ABB.NS', 'POWERINDIA.NS',
       'ACC.NS', 'ACCELYA.NS', 'ACCURACY.NS', 'ACEINTEG.NS', 'ACE.NS',
       'ADANIENT.NS', 'ADANIGREEN.NS', 'ADANIPORTS.NS', 'ADANIPOWER.NS',
       'ADANITRANS.NS', 'ADFFOODS.NS', 'ABCAPITAL.NS', 'ABFRL.NS',
       'BIRLAMONEY.NS', 'ADORWELD.NS', 'ADROITINFO.NS', 'ADVENZYMES.NS',
       'ADVANIHOTR.NS', 'AEGISCHEM.NS', 'AFFLE.NS', 'AGARIND.NS',
       'AGRITECH.NS', 'AGROPHOS.NS', 'ATFL.NS', 'AHLADA.NS',
       'AHLUCONT.NS'], dtype=object)

In [9]:
df.cols

Tax Effect Of Unusual Items               float64
Tax Rate For Calcs                        float64
Normalized EBITDA                         float64
Total Unusual Items                       float64
Total Unusual Items Excluding Goodwill    float64
                                           ...   
Selling And Marketing Expense             float64
Other Income Expense                      float64
Rent Expense Supplemental                 float64
Rent And Landing Fees                     float64
company_symbol                             object
Length: 70, dtype: object